### Load imports and run the no LID case:

In [4]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking model BEFORE adding GSI
%reset
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import runswmm
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'y16m10d11_demonstrate_runswmm_on_aws-2'
client = MongoClient(mongoServer)
swmmInpFile = "wingohocking_borg_initial.inp"  # initial SWMM .inp file
lidZeroFile = "wingo_Wakefield_list_zero.yaml"
################################################

db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.
0.0 min, 2.35 sec
volume = 319.424
Stored record 57fd6438c10157ceea9ce93b for zero LID usage:  peak = None, volume = 319.424


### Loop ALL subcatchment LID numbers over a range:

In [5]:
# Now, apply a range of Wakefield Park rain gardens in all seven subcatchments simultaneously
from copy import deepcopy
for numberForAllLid in range(1,21):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

0.0 min, 4.66 sec
volume = 318.733
Stored record 57fd647dc10157ceea9ce93c for volume = 318.733
volumeReduction = 0.691
0.0 min, 5.09 sec
volume = 318.042
Stored record 57fd6482c10157ceea9ce93d for volume = 318.042
volumeReduction = 1.382
0.0 min, 4.60 sec
volume = 317.351
Stored record 57fd6486c10157ceea9ce93e for volume = 317.351
volumeReduction = 2.073
0.0 min, 4.72 sec
volume = 316.661
Stored record 57fd648bc10157ceea9ce93f for volume = 316.661
volumeReduction = 2.763
0.0 min, 4.45 sec
volume = 315.971
Stored record 57fd6490c10157ceea9ce940 for volume = 315.971
volumeReduction = 3.453
0.0 min, 4.14 sec
volume = 315.282
Stored record 57fd6494c10157ceea9ce941 for volume = 315.282
volumeReduction = 4.142
0.0 min, 4.17 sec
volume = 314.594
Stored record 57fd6498c10157ceea9ce942 for volume = 314.594
volumeReduction = 4.83
0.0 min, 4.08 sec
volume = 313.906
Stored record 57fd649cc10157ceea9ce943 for volume = 313.906
volumeReduction = 5.518
0.0 min, 4.09 sec
volume = 313.218
Stored record 

### To display all of the runs from the database:

In [6]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

{u'lidDict': None, u'swmmStartTime': u'2016-10-11 18:02:01.124736', u'swmmInputFileStr': u'[TITLE]\nWingohocking\n[OPTIONS]\nFLOW_UNITS\tCFS \nINFILTRATION\tHORTON \nFLOW_ROUTING\tKINWAVE \nLINK_OFFSETS\tDEPTH \nMIN_SLOPE\t0 \nALLOW_PONDING\tNO \nSKIP_STEADY_STATE\tNO \nSTART_DATE\t01/01/2008 \nSTART_TIME\t00:00:00 \nREPORT_START_DATE\t01/01/2008 \nREPORT_START_TIME\t00:00:00 \nEND_DATE\t01/01/2009 \nEND_TIME\t00:00:00 \nSWEEP_START\t01/01 \nSWEEP_END\t12/31 \nDRY_DAYS\t0 \nREPORT_STEP\t00:01:00 \nWET_STEP\t00:01:00 \nDRY_STEP\t01:00:00 \nROUTING_STEP\t0:00:15 \nINERTIAL_DAMPING\tPARTIAL \nNORMAL_FLOW_LIMITED\tSLOPE \nFORCE_MAIN_EQUATION\tH-W \nVARIABLE_STEP\t0.75 \nLENGTHENING_STEP\t0 \nMIN_SURFAREA\t0 \nMAX_TRIALS\t0 \nHEAD_TOLERANCE\t0 \nSYS_FLOW_TOL\t5 \nLAT_FLOW_TOL\t5 \nMINIMUM_STEP\t0.5 \nTHREADS\t1 \n[EVAPORATION]\nTEMPERATURE\t\nDRY_ONLY\tNO \n[TEMPERATURE]\nFILE\t"668594.dat" \nWINDSPEED\tFILE \nSNOWMELT\t34 0.5 0.6 0.0 50.0 0.0 \nADC\tPERVIOUS 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0

### To Reconstitute the SWMM input file for any of the runs above

In [5]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()